***
## Реализация алгоритма в коде

Таблица, которая помогла решить задачу, в коде будет представлена в виде двумерного массива — например, списка списков.

![alt text](image_1710147487.png)

```py
containers = [
#    1кг   2кг    3кг   4кг
    ['Г',  'Г',  'Г',  'Г'],    # Первая строка таблицы.
    ['Г',  'Г',  'Г',  'М'],    # Вторая строка таблицы.
    ['Г',  'Г',  'Н',  'Н+Г'],  # Третья строка таблицы.
    ['И', 'И+Г', 'И+Г', 'И+Н']  # Четвёртая строка таблицы.
...
]
# Любую ячейку можно найти по двум индексам:
any_cell = containers[line][cell]
# line - это индекс строки, cell - индекс ячейки в строке.

# Например: cell = containers[1][3] содержит значение "М" - 
# это четвёртая по счёту ячейка во второй по счёту строке.
```

Вместимость «виртуальных контейнеров» — это индексы элементов в строке, значения индекса `cell`.

Все ячейки таблицы заполняются по одному и тому же правилу — значит, можно описать это правило для любой ячейки `containers[line][cell]`


Вес прибора из строки `[line]` обозначим как `cur_weight` (current weight, «текущий / актуальный вес»).

При наполнении очередного «виртуального контейнера» (очередной ячейки) нужно выбрать более выгодный набор оборудования (тот, что может провести наибольшее число экспериментов). Вариантов всего два:

* Набор 1:

    * взять набор из ячейки выше (из ячейки с индексом `containers[line - 1][cell]`);

* Набор 2:

    * положить в ячейку текущий прибор (его вес — `cur_weight`),

    * вычислить оставшееся место как `cell - cur_weight`,

    * если в ячейке осталось свободное место — добавить в текущую ячейку набор из ячейки `containers[line - 1][cell - cur_weight]`.

В итоге алгоритм сводится к сравнению «выгодности» первого и второго наборов.

Теперь можно написать программу, которая в каждую следующую ячейку вписывает или значение из предыдущей строки или новый прибор, к которому добавлено содержимое ячейки из предыдущей строки (если в текущей ячейке осталось свободное место).

Заполняем очередную ячейку в двумерном массиве:

![alt text](image_1710930987.png)

После внесения поправки алгоритм будет выглядеть так:

![alt text](image_1710930999.png)

В коде ход решения может быть таким:

* создать пустой двумерный массив нужного размера;

* по заданным правилам заполнить элементы (ячейки) значениями;

* вернуть результат — содержимое последнего элемента последнего вложенного массива.

In [4]:
from pprint import pprint


def backpack_problem_solution(
        tools: list[tuple[str, int, int]], capacity: int
) -> str:
    # Сохраняем количество приборов в переменную.
    items_count = len(tools)
    # Создаём таблицу. В каждой ячейке должны храниться число и список:
    # количество экспериментов и список приборов. Для простоты подсчётов 
    # добавим строку с отсутствием рассматриваемых приборов
    # и столбец с нулевой вместимостью контейнера.
    table = [
        [[0, []] for _ in range(capacity + 1)] for _ in range(items_count + 1)
    ]
    # Для каждого прибора:
    for row_number in range(1, items_count + 1):
        # Распаковываем кортеж с данными о приборе.
        name, mass, experiments = tools[row_number - 1]
        # Для контейнера вместимостью от 1 и до максимальной вместимости:
        for volume in range(1, capacity + 1):
            # Если вес прибора меньше или равен 
            # вместимости рассматриваемого контейнера.
            if mass <= volume:
                # Считаем количество экспериментов для текущего прибора
                # плюс наилучшее решение для оставшейся вместимости 
                # из предыдущей строки.
                total_experiments_with_current_tool = (
                    experiments + table[row_number - 1][volume - mass][0]
                )
                # Количество экспериментов 
                # в текущей колонке на предыдущей строке:
                previous_result = table[row_number - 1][volume][0]
                # Если результат с текущим прибором лучше:
                if total_experiments_with_current_tool > previous_result:
                    # Записываем количество экспериментов.
                    table[row_number][volume][0] = (
                        total_experiments_with_current_tool
                    )
                    # Копируем список приборов из предыдущей строки 
                    # из ячейки, равной оставшейся вместимости.
                    table[row_number][volume][1] = list.copy(
                        table[row_number - 1][volume - mass][1]
                    )
                    # Добавляем к списку приборов
                    # имя текущего прибора.
                    table[row_number][volume][1].append(name)
                else:
                    # Если результат с рассматриваемым прибором 
                    # хуже или такой же -
                    # переносим результат с предыдущей строки.
                    table[row_number][volume] = table[row_number - 1][volume]
            else:
                # Если масса рассматриваемого прибора 
                # больше вместимости ячейки - 
                # переносим результат с предыдущей строки.
                table[row_number][volume] = table[row_number - 1][volume]
    # Распечатываем таблицу для проверки.
    pprint(table)
    # Возвращаем строку с названиями приборов
    # из нижней правой ячейки таблицы, через запятую.
    return ', '.join(table[-1][-1][-1])    


if __name__ == '__main__':
    # [('название прибора', масса, количество экспериментов)]
    tools = [
        ('гигрометр', 1, 3),
        ('масс-спектрометр', 4, 6),
        ('нивелир', 3, 4),
        ('интерферометр', 1, 4)
    ]
    result = backpack_problem_solution(tools, 4)
    print(result)


[[[0, []], [0, []], [0, []], [0, []], [0, []]],
 [[0, []],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [3, ['гигрометр']]],
 [[0, []],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [6, ['масс-спектрометр']]],
 [[0, []],
  [3, ['гигрометр']],
  [3, ['гигрометр']],
  [4, ['нивелир']],
  [7, ['гигрометр', 'нивелир']]],
 [[0, []],
  [4, ['интерферометр']],
  [7, ['гигрометр', 'интерферометр']],
  [7, ['гигрометр', 'интерферометр']],
  [8, ['нивелир', 'интерферометр']]]]
нивелир, интерферометр


***
## И что в итоге?

Задача о рюкзаке — классический пример для динамического программирования. Эта задача вполне может встретиться на алгоритмических собеседованиях или олимпиадах. 

Сложность приведённого алгоритма напрямую зависит от длины входного массива и от «вместимости» рюкзака. В каждой строке выполняется определённое количество операций, и это количество операций зависит от вместимости.

В итоге сложность этого алгоритма — `O(mn)`, где `m` означает «вместимость рюкзака» — это вместимость контейнера (4 кг) в примере с приборами или общее доступное количество времени (3 часа) в примере с мёдом. 

Сравнительно с жадными алгоритмами и с методом полного перебора такая сложность — это чистая победа по всем параметрам.

* Решение задач методом динамического программирования начинается с определения подзадач.

* Подзадачи должны быть автономны: в результате решения одной подзадачи не должны изменяться условия других подзадач.

* Для последовательного решения подзадач обычно строится таблица.

Приведённый алгоритм — не единственный в динамическом программировании. Однако у алгоритмов динамического программирования есть основная общая черта: единожды вычисленное решение подзадачи сохраняется и применяется в дальнейших вычислениях — так содержимое заполненной ячейки применялось для дополнения одной из следующих полупустых ячеек в задачах о рюкзаке.

Алгоритмы динамического программирования применяются во многих практических задачах, например — при работе с текстом:

* при поиске максимальной общей строки в двух текстах;

* при поиске различий между двумя текстами;

* для проверки орфографии;

* для систем автозамены текста при опечатках…